In [1]:
#Import required libraries
import gspread
import sys
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import numpy as np
import urllib
import sqlalchemy
from gspread_dataframe import set_with_dataframe
from gspread_dataframe import get_as_dataframe

In [2]:
#Import project specific functions
from column_map import column_map
from yesno_functions import *

In [3]:
#Import shared functions
sys.path.append('../..')
from IPM_Shared_Code_public.Python.google_creds_functions import create_assertion_session
from IPM_Shared_Code_public.Python.utils import get_config
from IPM_Shared_Code_public.Python.delta_functions import *
from IPM_Shared_Code_public.Python.sql_functions import *

C:\ProgramData\Anaconda3\lib\site-packages\authlib\client\__init__.py:12: AuthlibDeprecationWarning: Deprecate "authlib.client", USE "authlib.integrations.requests_client" instead.
It will be compatible before version 1.0.
Read more <https://git.io/Jeclj#file-rn-md>
  deprecate('Deprecate "authlib.client", USE "authlib.integrations.requests_client" instead.', '1.0', 'Jeclj', 'rn')


### Use the config file to setup connections

In [4]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
server = config['srv']['server']
dwh = config['db']['crowdsdb']
cred_file = config['google']['path_to_file']

In [5]:
con_string = 'Driver={' + driver + '};Server=' + server +';Database=' + dwh + ';Trusted_Connection=Yes;'
params = urllib.parse.quote_plus(con_string)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

### Execute the function to get the renamed columns for this sheet

In [6]:
#Call the column map function to get the dictionary to be used for renaming and subsetting the columns
col_rename = column_map('patrol_dpr')

In [7]:
#Because of duplicate column names these columns are renamed based on the column index and the keys and 
#values need to be switched
col_rename = {v[0]: k for k, v in col_rename.items()}

In [8]:
cols = list(col_rename.values())

### Read the current data from SQL

In [13]:
sql = 'select * from crowdsdb.dbo.tbl_dpr_patrol'

In [14]:
patrol_sql = (pd.read_sql(con = engine, sql = sql)
              .drop(columns = ['patrol_id', 'patroncount'])
              .fillna(value = np.nan, axis = 1))

In [15]:
sql_cols = list(patrol_sql.columns.values)

In [16]:
patrol_sql.head()

,encounter_timestamp,encounter_datetime,site_id,location_adddesc,park_division,visit_reason,firstname_1,lastname_1,firstname_2,lastname_2,...,sd_patronscomplied,sd_patronsnocomply,sd_amenity,summonscount_a01,summonscount_a03,summonscount_a04,summonscount_a22,other_summonstype,other_summonscount,borough


In [17]:
hash_rows(patrol_sql, exclude_cols = ['encounter_timestamp'], hash_name = 'row_hash')

### Read the site reference list from SQL

In [18]:
sql = 'select * from crowdsdb.dbo.tbl_ref_sites'

In [19]:
site_ref = pd.read_sql(con = engine, sql = sql)[['site_id', 'site_desc', 'borough']]

### Read the latest data from Google Sheets

In [20]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name(cred_file, scope)
client = gspread.authorize(creds)

In [21]:
sheet = client.open('COMBINED Patrol Reporting Responses')

In [22]:
ws = sheet.worksheet('MASTER')

In [24]:
#Read the worksheet as a data frame, rename the columns and subset the columns to only include those
#in the column list
patrol = (get_as_dataframe(ws, evaluate_formulas = True, header= None)
          #Always remove row 0 with the column headers
          .iloc[1:]
          .rename(columns = col_rename)
          .fillna(value = np.nan, axis = 1))[cols]

In [25]:
patrol.head()

,encounter_timestamp,encounter_datetime,site_desc,location_adddesc,park_division,firstname_1,lastname_1,firstname_2,lastname_2,firstname_3,...,sd_pdassist,sd_pdcontact,sd_comments,summonscount_a01,summonscount_a03,summonscount_a04,summonscount_a22,other_summonstype,other_summonscount,borough
1,5/29/2020 17:25:27,5/29/2020 17:08:00,Van Cortlandt Park-Southwest Playground,NaN,Rangers,Brittany,Nulph,Ilana,Weinstein,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bronx
2,5/29/2020 17:24:51,5/29/2020 17:23:00,Young Park,NaN,Rangers,Hunter,Mathis,Albert,Seto,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bronx
3,5/29/2020 17:24:50,5/29/2020 16:49:00,Van Cortlandt Park-Woodlawn Playground,NaN,Rangers,Brittany,Nulph,Ilana,Weinstein,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bronx
4,5/29/2020 17:24:01,5/29/2020 16:40:00,Van Cortlandt Park-Sachkerah Woods Playground,NaN,Rangers,Brittany,Nulph,Ilana,Weinstein,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bronx
5,5/29/2020 17:23:27,5/29/2020 17:10:00,West Harlem Piers,NaN,PEP,Anna,Williams,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Manhattan


In [26]:
yesno = ['closed_education', 'closed_outcome', 'closed_summonsissued', 'closed_pdassist',
         'closed_pdcontact', 'sd_summonsissued', 'sd_pdassist', 'sd_pdcontact']

In [27]:
yesno_cols(patrol, yesno)

In [28]:
#Remove any rows with no data, presumably these are rows with no timestamp
patrol = patrol[patrol['encounter_timestamp'].notna()]

In [29]:
patrol = patrol.merge(site_ref, how = 'left', on = ['site_desc', 'borough'])[sql_cols]

In [30]:
hash_rows(patrol, exclude_cols = ['site_id', 'encounter_timestamp'], hash_name = 'row_hash')

In [31]:
patrol_deltas = (check_deltas(new_df = patrol, old_df = patrol_sql, on = ['site_id', 'encounter_timestamp'], 
                              hash_name = 'row_hash', dml_col = 'dml_verb'))[sql_cols + ['dml_verb']]

In [32]:
patrol_inserts = patrol_deltas[patrol_deltas['dml_verb'] == 'I'][sql_cols]

In [33]:
patrol_inserts.head()

,encounter_timestamp,encounter_datetime,site_id,location_adddesc,park_division,visit_reason,firstname_1,lastname_1,firstname_2,lastname_2,...,sd_patronscomplied,sd_patronsnocomply,sd_amenity,summonscount_a01,summonscount_a03,summonscount_a04,summonscount_a22,other_summonstype,other_summonscount,borough
0,5/29/2020 17:25:27,5/29/2020 17:08:00,X092-09,NaN,Rangers,Standard Patrol,Brittany,Nulph,Ilana,Weinstein,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bronx
1,5/29/2020 17:24:51,5/29/2020 17:23:00,X073,NaN,Rangers,Standard Patrol,Hunter,Mathis,Albert,Seto,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bronx
2,5/29/2020 17:24:50,5/29/2020 16:49:00,X092-06,NaN,Rangers,Standard Patrol,Brittany,Nulph,Ilana,Weinstein,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bronx
3,5/29/2020 17:24:01,5/29/2020 16:40:00,X092-26,NaN,Rangers,Standard Patrol,Brittany,Nulph,Ilana,Weinstein,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bronx
4,5/29/2020 17:23:27,5/29/2020 17:10:00,M376,NaN,PEP,Fixed Post,Anna,Williams,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Manhattan


In [35]:
patrol_inserts.to_sql('tbl_dpr_patrol', engine, index = False, if_exists = 'append')

In [36]:
patrol_updates = patrol_deltas[patrol_deltas['dml_verb'] == 'U'][sql_cols]

In [40]:
patrol_updates.head()

,encounter_timestamp,encounter_datetime,site_id,location_adddesc,park_division,visit_reason,firstname_1,lastname_1,firstname_2,lastname_2,...,sd_patronscomplied,sd_patronsnocomply,sd_amenity,summonscount_a01,summonscount_a03,summonscount_a04,summonscount_a22,other_summonstype,other_summonscount,borough


In [41]:
sql_update(patrol_updates, 'tbl_dpr_patrol', engine, ['encounter_timestamp', 'site_id'])